# Redis
### Store cars data

#### Install needed to run Redis

In [ ]:
!pip install redis

In [ ]:
#!pip install flask-bootstrap

#### Preparing the data

In [ ]:
import pandas as pd
import pickle

In [ ]:
df_carList = pd.read_csv("./datasets/Car_listings.csv", delimiter=",")

We can see that this dataset has a Price column, which we don't need, beacuse this is a rental company where customers doesn't buy the cars. Because of this we remove the Price column:

In [ ]:
df_carList = df_carList.drop(['Price'], axis = 1)
df_carList.head(1)

In [ ]:
df_carList = df_carList.sample(frac = 1)

#### We need to convert the csv file to JSON, so that we can store the data in redis

In [ ]:
df_carList.to_json('./datasets/cars.json', orient='split', compression='infer', index='true')

In [ ]:
df = pd.read_json('./datasets/cars.json', orient ='split', compression = 'infer')

In [ ]:
df

#### Redis

In [9]:
import redis
import json
import time

In [11]:
r = redis.Redis(db=1)
#r.flushdb()

In [ ]:
with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)
 
    # Print the type of data variable
    #print("Type:", type(data))
    
    #print("ITEMS: ", data.items())
      
    # Print the data of dictionary
    #print("\nData:", data['data'][0]['ID IPEDS Occupation Parent'])
    print("\nData:", data['data'][0])

In [ ]:
# Loop og give hver værdi en key som er deres kolonne navn
columns = list(df)
listOfDict = []

def smallDictFunction(co: [], di: {}) -> {}:
    smallDicts = {}
    for c in co:
        for d in di:
            if (columns.index(c) == dic.index(d)):
                #print(c + ': ' + str(d))
                #print(d)
                smallDicts[c] = d
        
    return smallDicts
   

for dic in data['data']:
    listOfDict.append(smallDictFunction(columns, dic))
    if (data['data'].index(dic)+1 == 200000):
        break    # break here


#print(test())
#print(smallDicts)
print(listOfDict)
#print(fullDict)

In [ ]:
index_list = [*range(1, (len(listOfDict)+1), 1)]
len(index_list)

In [ ]:
zipped = zip(index_list, listOfDict)
fullDict = dict(zipped)
print(fullDict)

In [ ]:
with open("./datasets/carsDict.json", "w") as write_file:
    json.dump(fullDict, write_file, indent=4)

In [ ]:
with open('./datasets/carsDict.json') as json_file:
    data = json.load(json_file)

In [ ]:
for dic in data:
    p_mydict = pickle.dumps(data[dic])
    r.set(dic,p_mydict)

In [ ]:
read_dict = r.get(200000)
yourdict = pickle.loads(read_dict)
yourdict

In [ ]:
%%writefile redisFile.py

from flask import Flask, request, jsonify, render_template
import redis
import json
import pickle

app = Flask(__name__, template_folder='hmtl_documents')


r = redis.Redis(db=1)
#r.flushdb()

with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)

read_dict = r.get(200000)
yourdict = pickle.loads(read_dict)
print(yourdict)

def getData2(n):
    dataList = []
    tic = time.perf_counter()
    d = r.get(n)
    yourdict = pickle.loads(d)
    dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

@app.route('/')
def start():
    #attributes = getAttributes()
    attributes = ['1', '2']
    return render_template('start.html', attributes=yourdict, time=attributes[-1])

if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
!python redisFile.py

#### Postgres
Vi skal kunne søge i postgres efter en person, lave den person inclussiv addresse om til dict og smide den i mongo i samme dict som redis

In [ ]:
!pip install psycopg2

In [4]:
from configparser import ConfigParser
import psycopg2

In [ ]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="postgres")

In [5]:
def config(filename='../Postgresql/postgresDB.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [117]:

def createDict(infoTuple, columnList, index):
    infoList = []
    for i in infoTuple:
        infoList.append(i)
        if (infoTuple.index(i) == index):
            break;
    return(dict(zip(columnList, infoList)))

fullUserDict= {}
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)
		
        # create a cursor
        cur = conn.cursor()
        
	# execute a statement
        print('Retrieving data:')
        print('--------------')
        #Retrieving data
        cur.execute('SELECT * from users where user_id = '+str(208))
        user = cur.fetchone()
        cur.execute('SELECT role from roles where role_id = '+str(user[6]))
        role = cur.fetchone()
        # Saving the user as a dict
        userTableColumns = ['user_id', 'firstName', 'lastName', 'age', 'email']
        userDict = createDict(user, userTableColumns, 4)
        userDict["role"] = role[0]
        print(userDict)
        print('--------------')
        
        cur.execute('SELECT address_id from users where user_id = '+str(208))
        address_id = cur.fetchone()
        cur.execute('SELECT * from address where address_id = '+str(address_id[0]))
        address = cur.fetchone()
        zipcode = address[2]
        cur.execute('SELECT * from cities where zipcode = '+str(zipcode))
        city = cur.fetchone()
        addressList = list(address)
        addressList.append(city[1])
        
        addressTableColumns = ['address_id', 'address', 'zipcode', 'city']
        addressDict = createDict(addressList, addressTableColumns, 3)
        print(addressDict)
        print('--------------')
        
        fullUserDict = userDict
        fullUserDict['address'] = addressDict
        
       
	# close the communication with the PostgreSQL
        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')
            #print(fullUserDict)
            return fullUserDict
            


if __name__ == '__main__':
    connect()
    
fullUserDict = connect()
print(fullUserDict)

Connecting to the PostgreSQL database...
Retrieving data:
--------------
{'user_id': 208, 'firstName': 'Alisa', 'lastName': 'Mason', 'age': 20, 'email': 'a.mason@randatmail.com', 'role': 'user'}
--------------
{'address_id': 7, 'address': 'Sønderstræde 88', 'zipcode': 2450, 'city': 'København SV'}
--------------
Database connection closed.
Connecting to the PostgreSQL database...
Retrieving data:
--------------
{'user_id': 208, 'firstName': 'Alisa', 'lastName': 'Mason', 'age': 20, 'email': 'a.mason@randatmail.com', 'role': 'user'}
--------------
{'address_id': 7, 'address': 'Sønderstræde 88', 'zipcode': 2450, 'city': 'København SV'}
--------------
Database connection closed.
{'user_id': 208, 'firstName': 'Alisa', 'lastName': 'Mason', 'age': 20, 'email': 'a.mason@randatmail.com', 'role': 'user', 'address': {'address_id': 7, 'address': 'Sønderstræde 88', 'zipcode': 2450, 'city': 'København SV'}}


#### MongoDB

In [ ]:
!pip install pymongo

In [7]:
import pymongo
import bson
import pickle
import base64

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

#use database named "organisation"
mydb = myclient["ExamProject"]

#use collection named "developers"
mycol = mydb["Temporary"]

In [118]:
def getData2(n1, n2):
    dataList = []
    tic = time.perf_counter()
    for n in range(int(n1), int(n2)+1):
        #print(n)
        #print(r.hgetall(str(n)))
        d = r.get(n)
        yourdict = pickle.loads(d)
        dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

def getData(n):
    dataList = []
    tic = time.perf_counter()
    d = r.get(n)
    yourdict = pickle.loads(d)
    dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

car = getData(1)
car[0]

{'Year': 2014,
 'Mileage': 35725,
 'City': 'El Paso',
 'State': ' TX',
 'Vin': '19VDE2E53EE000083',
 'Make': 'Acura',
 'Model': 'ILX6-Speed'}

In [119]:
booking = {}
booking['user'] = fullUserDict
booking['car'] = car[0]

print(booking)

{'user': {'user_id': 208, 'firstName': 'Alisa', 'lastName': 'Mason', 'age': 20, 'email': 'a.mason@randatmail.com', 'role': 'user', 'address': {'address_id': 7, 'address': 'Sønderstræde 88', 'zipcode': 2450, 'city': 'København SV'}}, 'car': {'Year': 2014, 'Mileage': 35725, 'City': 'El Paso', 'State': ' TX', 'Vin': '19VDE2E53EE000083', 'Make': 'Acura', 'Model': 'ILX6-Speed'}}


In [120]:
x = mycol.insert_one(booking)

In [121]:
for x in mycol.find():
    print(x)
    print('------------------')

{'_id': ObjectId('628dbb5deedb38463899a421'), 'user': {'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user', 'address': {'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}}, 'car': {'Year': 2017, 'Mileage': 15630, 'City': 'Katy', 'State': ' TX', 'Vin': '1FMJU1JT4HEA55066', 'Make': 'Ford', 'Model': 'ExpeditionXLT'}}
------------------
{'_id': ObjectId('628dd5e5ca6d87335a883c75'), 'user': {'user_id': 405, 'firstName': 'Belinda', 'lastName': 'Foster', 'age': 44, 'email': 'b.foster@randatmail.com', 'role': 'user', 'address': {'address_id': 9, 'address': 'Kongshøj Allé 65', 'zipcode': 4220, 'city': 'Korsør'}}, 'car': {'Year': 2014, 'Mileage': 35725, 'City': 'El Paso', 'State': ' TX', 'Vin': '19VDE2E53EE000083', 'Make': 'Acura', 'Model': 'ILX6-Speed'}}
------------------
{'_id': ObjectId('628df1b3ca6d87335a883c76'), 'user': {'user_id': 208, 'firstName': 'Alisa', 'lastName': 'Mason', 'age': 

#### Neo4j

In [15]:
!pip install neo4j

In [17]:
import logging
import sys

from neo4j import GraphDatabase
from neo4j.exceptions import ServiceUnavailable

# DENNE

In [122]:
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password), database='analisys')

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    @staticmethod
    def enable_log(level, output_stream):
        handler = logging.StreamHandler(output_stream)
        handler.setLevel(level)
        logging.getLogger("neo4j").addHandler(handler)
        logging.getLogger("neo4j").setLevel(level)

    def create_friendship(self, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_friendship, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books)
            for row in result:
                print("Created friendship between: {u}, {c} from {books}"
                      .format(
                          u=row['u'],
                          c=row['c'],
                          books=row['books']))

    @staticmethod
    def _create_and_return_friendship(tx, user_id, 
                      firstName, 
                      lastName,
                      age,
                      email,
                      address,
                      zipcode,
                      city,
                      year,
                      mileage,
                      cityC,
                      state,
                      vin,
                      make,
                      model, books):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        query = (
            "MERGE (u:User { user_id: $user_id, firstName: $firstName, lastName: $lastName, age: $age, email: $email, address: $address, zipcode: $zipcode, city: $city}) "
            "MERGE (c:Car { year: $year, mileage: $mileage, cityC: $cityC, state: $state, vin: $vin, make: $make, model: $model }) "
            "MERGE (u)-[k:BOOKS { from: $books }]->(c) "
            "RETURN u, c, k"
        )
        result = tx.run(query, user_id=user_id, firstName=firstName, lastName=lastName, age=age, email=email, 
                        address=address, zipcode=zipcode, city=city, year=year, mileage=mileage, cityC=cityC, state=state,
                        vin=vin, make=make, model=model, books=books)
        try:
            return [{
                        "u": row["u"]["user_id"],
                        "c": row["c"]["year"],
                        "books": row["k"]["books"]
                    }
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise


if __name__ == "__main__":
    bolt_url = "bolt://localhost:7687"
    user = "neo4j"
    password = "1234"
    App.enable_log(logging.INFO, sys.stdout)
    app = App(bolt_url, user, password)
    
    infoList = []
    for x in mycol.find():
        #print(list(x.values())[1])
        for key, values in x.items():
            if (key == 'user'):
                #x = thisdict["model"]
                listVU = list(values)
                #print(values)

                for k, v in values.items():
                    infoList.append(v)
                infoList.pop()
                infoList.pop()
                for k, v in values['address'].items():
                    infoList.append(v)
                del infoList[5]
                #print(listVU)
                print('--------------')
            if (key == 'car'):
                print(values)
                for k, v in values.items():
                    infoList.append(v)
            #print('key: ' + key)
            #print('values: ' + str(values))
        print(infoList)
        app.create_friendship(infoList[0], infoList[1], infoList[2], infoList[3], infoList[4], infoList[5], 
                              infoList[6], infoList[7], infoList[8], infoList[9], infoList[10], infoList[11], 
                              infoList[12], infoList[13], infoList[14], 'books')
        print('------------------')
        infoList = []
    
    #app.create_friendship(405, 'Belinda', 'Foster', 44, 'b.foster@randatmail.com', 'Kongshøj Allé 65', 4220, 'Korsør',
                         #2017, 15630, 'Katy', 'TX', '1FMJU1JT4HEA55066', 'Ford', 'ExpeditionXLT', 'books')
    #app.find_person("Alice")
    
    app.close()

--------------
{'Year': 2017, 'Mileage': 15630, 'City': 'Katy', 'State': ' TX', 'Vin': '1FMJU1JT4HEA55066', 'Make': 'Ford', 'Model': 'ExpeditionXLT'}
[405, 'Belinda', 'Foster', 44, 'b.foster@randatmail.com', 'Kongshøj Allé 65', 4220, 'Korsør', 2017, 15630, 'Katy', ' TX', '1FMJU1JT4HEA55066', 'Ford', 'ExpeditionXLT']
Created friendship between: 405, 2017 from None
------------------
--------------
{'Year': 2014, 'Mileage': 35725, 'City': 'El Paso', 'State': ' TX', 'Vin': '19VDE2E53EE000083', 'Make': 'Acura', 'Model': 'ILX6-Speed'}
[405, 'Belinda', 'Foster', 44, 'b.foster@randatmail.com', 'Kongshøj Allé 65', 4220, 'Korsør', 2014, 35725, 'El Paso', ' TX', '19VDE2E53EE000083', 'Acura', 'ILX6-Speed']
Created friendship between: 405, 2014 from None
------------------
--------------
{'Year': 2014, 'Mileage': 35725, 'City': 'El Paso', 'State': ' TX', 'Vin': '19VDE2E53EE000083', 'Make': 'Acura', 'Model': 'ILX6-Speed'}
[208, 'Alisa', 'Mason', 20, 'a.mason@randatmail.com', 'Sønderstræde 88', 2450

In [ ]:
%%writefile application2.py

from flask import Flask, request, jsonify, render_template
import redis
import json
import time
import pandas as pd
import pickle
from redisFile import getData2
#import redisFile.py as redisFile

app = Flask(__name__, template_folder='hmtl_documents')

# ---------------- THE CODE -------------------------
df_carList = pd.read_csv("./datasets/Car_listings.csv", delimiter=",")
df_carList = df_carList.drop(['Price'], axis = 1)

df_carList.to_json('./datasets/cars.json', orient='split', compression='infer', index='true')
df = pd.read_json('./datasets/cars.json', orient ='split', compression = 'infer')

r = redis.Redis(db=1)

with open('./datasets/cars.json') as json_file:
    data = json.load(json_file)
    
# Loop og give hver værdi en key som er deres kolonne navn
columns = list(df)
listOfDict = []

def smallDictFunction(co: [], di: {}) -> {}:
    smallDicts = {}
    for c in co:
        for d in di:
            if (columns.index(c) == dic.index(d)):
                smallDicts[c] = d
    return smallDicts
   
for dic in data['data']:
    listOfDict.append(smallDictFunction(columns, dic))
    if (data['data'].index(dic)+1 == 5):
        break    # break here

index_list = [*range(1, (len(listOfDict)+1), 1)]
fullDict = dict(zip(index_list, listOfDict))

with open("./datasets/carsDict.json", "w") as write_file:
    json.dump(fullDict, write_file, indent=4)

with open('./datasets/carsDict.json') as json_file:
    data = json.load(json_file)

for dic in data:
    p_mydict = pickle.dumps(data[dic])
    r.set(dic,p_mydict)

def getData3(n):
    dataList = []
    tic = time.perf_counter()
    d = r.get(n)
    yourdict = pickle.loads(d)
    dataList.append(yourdict)
    toc = time.perf_counter()
    dataList.append(toc - tic)
    return dataList

def getAttributes():
    tic = time.perf_counter()
    unPrettyList = list(r.get('1'))
    toc = time.perf_counter()
    prettyList = []
    for s in unPrettyList:
        txt = str(s).split("'")
        prettyList.append(txt[1])
    time2 = toc - tic
    prettyList.append(time2)
    return prettyList

def getBySpecificAtt(n, s):
    tic = time.perf_counter()
    string = r.get(n,s)
    toc = time.perf_counter()
    time2 = toc - tic
    txt = str(string).split("'")
    
    dataList = []
    dataList.append(txt[1])
    dataList.append(time2)
    return dataList


# ---------------- THE ENDPOINTS --------------------
@app.route('/')
def start():
    #attributes = getAttributes()
    attributes = ['1', '2']
    return render_template('start.html', attributes=attributes[1:-1], time=attributes[-1])

@app.route('/data', methods=['GET', 'POST'])
def data():
    if request.method == 'POST':
        n = request.form['n']
        data = getData2(n)
          
        return render_template("data.html", data=data[int(n)], time=data[-1])
    
@app.route('/specificData', methods=['GET', 'POST'])
def specificData():
    if request.method == 'POST':
        n = request.form['n']
        s = request.form['s']
        data = getBySpecificAtt(n, s)
          
        return render_template("specificData.html", data=data[0], time=data[1])

@app.route('/redis', methods=['GET', 'POST'])
def predict():
    if request.method == 'GET':
        data = request.data
    return render_template('redis.html', carsDict=data)

if __name__ == '__main__':
    app.run(debug=True)

In [2]:
%%writefile application3.py

from flask import Flask, request, jsonify, render_template

app = Flask(__name__, template_folder='hmtl_documents')

@app.route('/')
def start():
    return render_template('start2.html')

if __name__ == '__main__':
    app.run(debug=True)

Writing application3.py


http://localhost:5000/

In [3]:
!python application3.py

^C
